In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch
import monai
import h5py
import utils
import image_utils
from skimage import transform
import random as rand
import nibabel as nib


def normalize_img(img):
    norm_img = img/np.max(img)
    return norm_img


def crop_pad_resize(image, nx, ny):
    x, y = image.shape

    x, y = slice.shape

    # difference in nr of pixels (divide by 2 since we have 2 sides)
    x_s = (x - nx) // 2
    y_s = (y - ny) // 2
    x_c = (nx - x) // 2
    y_c = (ny - y) // 2

    if x > nx and y > ny:
        # if image is larger in both dimensions cut a slice
        slice_cropped = slice[x_s:x_s + nx, y_s:y_s + ny]

    else:
        # if one dim is smaller fill that side up with 0's
        slice_cropped = np.zeros((nx, ny))

        if x <= nx and y > ny:
            # fill up x direction with 0's, cut in x direction
            slice_cropped[x_c:x_c + x, :] = slice[:, y_s:y_s + ny]
        elif x > nx and y <= ny:
            # fill up y direction with 0's, cut in y direction
            slice_cropped[:, y_c:y_c + y] = slice[x_s:x_s + nx, :]
        else:
            # if dimensions are as desired, keep the original slice
            slice_cropped[x_c:x_c + x, y_c:y_c + y] = slice[:, :]

    return slice_cropped


target_resolution = (1.36719, 1.36719)
nx, ny = (212, 212)
input_folder = 'data'
img_list = []
mask_list = []

for folder in os.listdir(input_folder):

    folder_path = os.path.join(input_folder, folder)

    for file in glob.glob(os.path.join(folder_path, 'patient???_frame??.nii.gz')):

        file_base = file.split('.nii.gz')[0]
        file_mask = file_base + '_gt.nii.gz'

        img_nii = nib.load(file)
        img_dat = img_nii.get_fdata()

        mask_nii = nib.load(file_mask)
        mask_dat = mask_nii.get_fdata()

        img = img_nii.get_fdata()
        mask = mask_nii.get_fdata()

        pixel_size = (img.shape[0], img.shape[1])

        scale_vector = [pixel_size[0] / target_resolution[0], pixel_size[1] / target_resolution[1]]

        for zz in range(img.shape[2]):
            # slice_img = normalize_img(np.squeeze(img[:, :, zz]))
            slice_img = np.squeeze(img[:, :, zz])
            img_rescaled = transform.rescale(slice_img,
                                             scale_vector,
                                             order=1,
                                             preserve_range=True,
                                             mode='constant')

            # slice_mask = normalize_img(np.squeeze(mask[:, :, zz]))
            slice_mask = np.squeeze(mask[:, :, zz])
            mask_rescaled = transform.rescale(slice_mask,
                                              scale_vector,
                                              order=0,
                                              preserve_range=True,
                                              mode='constant')

            # img_cropped = crop_pad_resize(img_rescaled, nx, ny)
            # mask_cropped = crop_pad_resize(mask_rescaled, nx, ny)

            plt.imshow(img_rescaled, 'gray')
            # img_list.append(img_rescaled)
            # mask_list.append(mask_rescaled)
print(img_list)